In [1]:
import math
from mlagents.plugins.skeleton_aware_op.skeleton import SkeletonLinear
import numpy as np

from mlagents.torch_utils import torch, default_device
from skeleton import SkeletonConv, SkeletonPool, SkeletonUnpool
from autoencoder import Encoder, StaticEncoder
from mlagents.plugins.bvh_utils import BVH_mod as BVH
from mlagents.plugins.bvh_utils.Quaternions import Quaternions 

In [2]:
file_path = "./data/Aj/walking.bvh"
# file_path2 = "./data/Aj/walking.bvh"
anim, names, frametime = BVH.load(file_path, need_quater = False)

batch_size = 2

# anim rotation shape : [motion_lenght, num_joints, 3]
# convert euler rotations to quaternions and concatenate them
rotations = anim.rotations[:batch_size,:,:]
glob_position = anim.positions[:batch_size, 0, :]
rotations = Quaternions.from_euler(np.radians(rotations)).qs

print("rotation shape : {}, glob pos shape : {}".format(rotations.shape, glob_position.shape))


rotation shape : (2, 23, 4), glob pos shape : (2, 3)


In [3]:

num_joints = len(anim.parents)
edges = np.zeros((num_joints-1, 2), dtype=int)

# generate edge list
count = 0
for i, parent in enumerate(anim.parents):
    # check if parent is existent
    if parent != -1:
        edges[count, 0] = i
        edges[count, 1] = parent
        count += 1


# need to invert the edge column order for it to work in pooling
edges[:,[0,1]] = edges[:,[1,0]]

# generate edge matrix by going through the edges and adding a one accordingly
edge_mat = np.eye(num_joints, dtype=int)
for edge in edges:
    edge_mat[edge, edge[::-1]] = 1

print(edges.shape)

(22, 2)


In [4]:
# reshape rotations so that we can add the index
index = []
for e in edges:
    index.append(e[0])
rotations = rotations[:,index, :]

print(rotations.shape)

# concatenate the rotations and positions + one 0 for padding
# rotations_cat = np.concatenate(rotations,axis=1)
rotations_cat = rotations.reshape(batch_size, -1)

print(rotations_cat.shape)

rotations_cat = np.concatenate([rotations_cat, glob_position.reshape(batch_size, -1), np.zeros((batch_size,1))], axis=1)

print(rotations_cat.shape)

# create a neighbour list
neighbour_list = []
for i in range(num_joints):
    neighbour_list.append(np.where(edge_mat[i] == 1)[0]) 

(2, 22, 4)
(2, 88)
(2, 92)


In [5]:
# Generate test layers of skeleton operations
conv_layer = SkeletonConv(neighbour_list, 23*4, 23*8, 23, bias=True, add_offset=False, in_offset_channel=0)
conv_offset_layer = SkeletonConv(neighbour_list, 23*4, 23*8, 23, bias=True, add_offset=True, in_offset_channel=3)
pool_layer = SkeletonPool( edges, 'mean', 8, last_pool=False)
unpool_layer = SkeletonUnpool( pool_layer.pooling_list, 8)
linear_layer = SkeletonLinear( neighbour_list, 23*3, 23*8)

[UNPOOLING] output edge number : 23


In [6]:
rotations_cat = torch.tensor(rotations_cat, requires_grad=False).float()
print("[INPUT] input shape :", rotations_cat.shape)
result = conv_layer.forward(rotations_cat)
print("[CONV] conv shape :",result.shape)

result_off = conv_layer.forward(rotations_cat)
print("[CONV] conv offset shape :",result.shape)

result = pool_layer.forward(result)
print("[POOL] pool shape :",result.shape)

offset_concat  = np.concatenate(anim.offsets, axis=0)[np.newaxis,:] 
linear_res = linear_layer.forward(torch.tensor(offset_concat).float()) 
print("[LINEAR] offset shape : ",offset_concat.shape)


result = unpool_layer.forward(result)
print("[UNPOOL] unpool shape :",result.shape)

# encoder = Encoder(edges)
# print("dim test:", rotations_cat.dim())x
# offset_concat  = np.concatenate(anim.offsets, axis=0)[np.newaxis,:] 
# print("offset shape : ",offset_concat.shape)
# latent = encoder.forward(rotations_cat, offset=torch.tensor(offset_concat).float())

# print("[ENCODER] latent shape :", latent.shape)
# print(result)

# edge_mat2 = calc_edge_mat(edges)
# print(edge_mat2)

[INPUT] input shape : torch.Size([2, 92])
[CONV] conv shape : torch.Size([2, 184])
[CONV] conv offset shape : torch.Size([2, 184])
[POOL] input shape : torch.Size([2, 184])
[POOL] self.weight.shape : torch.Size([96, 184])
[POOL] pool shape : torch.Size([2, 96])
[LINEAR] input shape:  torch.Size([1, 69])
[LINEAR] input reshape:  torch.Size([1, 69])
[LINEAR] res shape : torch.Size([1, 184])
[LINEAR] offset shape :  (1, 69)
[UNPOOL] unpool shape : torch.Size([2, 184])


In [7]:
torch.tensor(offset_concat).float().shape
print(anim.offsets.shape)

(23, 3)


In [8]:
staticEncoder = StaticEncoder(edges)
output = staticEncoder.forward(torch.tensor(anim.offsets[np.newaxis, :,:]).float())
for out in output:
    print(out.shape)

(22, 2)
[LINEAR] input shape:  torch.Size([1, 23, 3])
[LINEAR] input reshape:  torch.Size([1, 69])
[LINEAR] res shape : torch.Size([1, 138])
[POOL] input shape : torch.Size([1, 138])
[POOL] self.weight.shape : torch.Size([72, 138])
[LINEAR] input shape:  torch.Size([1, 72])
[LINEAR] input reshape:  torch.Size([1, 72])
[LINEAR] res shape : torch.Size([1, 144])
torch.Size([1, 23, 3])
torch.Size([1, 72])
torch.Size([1, 144])


In [9]:
staticEncoder = StaticEncoder(edges)
offsets = staticEncoder.forward(torch.tensor(anim.offsets[np.newaxis, :,:]).float())
for offset in offsets:
    print(offset.shape)

(22, 2)
[LINEAR] input shape:  torch.Size([1, 23, 3])
[LINEAR] input reshape:  torch.Size([1, 69])
[LINEAR] res shape : torch.Size([1, 138])
[POOL] input shape : torch.Size([1, 138])
[POOL] self.weight.shape : torch.Size([72, 138])
[LINEAR] input shape:  torch.Size([1, 72])
[LINEAR] input reshape:  torch.Size([1, 72])
[LINEAR] res shape : torch.Size([1, 144])
torch.Size([1, 23, 3])
torch.Size([1, 72])
torch.Size([1, 144])


In [11]:
encoder = Encoder(edges)
enc_output = encoder.forward(rotations_cat, offsets)
print(enc_output.shape)

[LINEAR] input shape:  torch.Size([1, 69])
[LINEAR] input reshape:  torch.Size([1, 69])
[LINEAR] res shape : torch.Size([1, 184])
[POOL] input shape : torch.Size([2, 184])
[POOL] self.weight.shape : torch.Size([96, 184])
[LINEAR] input shape:  torch.Size([1, 72])
[LINEAR] input reshape:  torch.Size([1, 72])
[LINEAR] res shape : torch.Size([1, 192])
[POOL] input shape : torch.Size([2, 192])
[POOL] self.weight.shape : torch.Size([112, 192])
torch.Size([2, 112])
